In [1]:
from openfermion.hamiltonians import MolecularData, load_molecular_hamiltonian
from openfermion.ops import FermionOperator
from openfermion.transforms import bravyi_kitaev
from openfermion.utils import hermitian_conjugated
from openfermion.ops import QubitOperator
from forestopenfermion import pyquilpauli_to_qubitop, qubitop_to_pyquilpauli
from pyquil.paulis import PauliSum
from scipy.optimize import minimize
from pyquil import Program
from pyquil.gates import *

import numpy as np
import functools

In [2]:
from pyquil import get_qc
qc = get_qc('4q-qvm')

In [9]:
numQubit = 4
trials = 1000
class Track:
    def __init__(self):
        self.cnt = 0

def solve_vqe(hamiltonian: PauliSum, numLayer) -> float:
    # Construct a variational quantum eigensolver solution to find the lowest
    # eigenvalue of the given hamiltonian
    
    #theta_init = np.random.rand(2*numQubit*numLayer) * 2 * np.pi
    theta_init = [ 6.28318850e+00,  2.69627327e+00,  3.64859364e-06,  1.71306484e+00,2.00379986e+00,  2.87101111e+00,  2.97499472e+00, -6.08056399e-04,2.93184856e+00,  2.20587629e+00,  3.14159630e+00,  1.78474312e+00,3.14173900e+00,  1.79790161e+00,  6.44982876e+00,  1.07299533e+00,6.28317088e+00,  6.62561448e-01,  3.14155614e+00,  5.39277594e+00,1.13776729e+00,  3.03874155e+00, -1.87878622e-04,  6.79330935e+00]
    track = Track()

    def inc(t, xk):
        t.cnt += 1
        
    def build_ansatz(theta_vec):
        a = None
        a = Program()
        for j in range(numLayer):
            for i in range(numQubit):
                a += RX(theta_vec[2*numQubit * j + 2 * i], i)
                a += RZ(theta_vec[2*numQubit * j + 2 * i + 1], i)
            for i in range(numQubit - 1):
                a += CNOT(i, i + 1)
        return a

    def ansatz_energy(t, theta_vec):
        t.cnt += 1
        energy = 0
        ansatz = None
        ansatz = build_ansatz(theta_vec)

        for term in hamiltonian:
            p = None
            p = Program()
            p += ansatz

            coeff = term.coefficient.real    # coefficient is real number in complex format
            opset =  term.operations_as_set()
            qubits_meas = []
            for op in opset:
                if op[1]=='X':
                    p += H(op[0])
                elif op[1]=='Y':
                    p += RZ(np.pi/2, op[0])
                    p += RY(np.pi/2, op[0])
                    p += RZ(-np.pi/2, op[0])
                elif op[1]=='Z':
                    pass
                qubits_meas.append(op[0])
                
            result = qc.run_and_measure(p, trials=trials)
            # measure the expectation value of eigenvalue in Z..Z basis. (number of 1 - number of -1)/trials
            eigval = 0
            for i in range(trials):
                tmp = 1
                for j in qubits_meas:
                    if result[j][i] == 0:
                        pass
                    else:
                        tmp = -tmp
                eigval += tmp
            energy += coeff*float(eigval)/trials
            
        print(energy)
        return energy

    theta_answer = minimize(functools.partial(ansatz_energy, track), theta_init, tol=1e-2).x
    return ansatz_energy(track, theta_answer), track.cnt

In [10]:
def get_ground_energy(interaction_hamil, numLayer):
    fermionop_hamil = FermionOperator()
    for key in interaction_hamil:
        value = interaction_hamil[key]
        fermionop_hamil += FermionOperator(term=key, coefficient=value)
        
    qubitop_hamil = bravyi_kitaev(fermionop_hamil)
    pauliop_hamil = qubitop_to_pyquilpauli(qubitop_hamil)

    return solve_vqe(pauliop_hamil, numLayer)


In [11]:
basis = 'sto-3g'
multiplicity = 1  # 2S+1
charge = 0

import matplotlib.pyplot as plt
%matplotlib inline

bond_lengths = [0.7] # np.linspace(0.3, 2.5, 23)
vqe_ground_energies_mean = []
vqe_ground_energies_min = []
vqe_ground_energies_std = []
vqe_iter_mean = []
vqe_iter_std = []

for layer in [3]:
    gelist = []
    gelist2 = []
    gestdlist = []
    iterlist = []
    iterstdlist = []
    for bond_length in bond_lengths:
        geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]
        description = str(round(bond_length, 2))
        h2_interaction_hamil = load_molecular_hamiltonian(geometry,
            basis,
            multiplicity,
            description,
            n_active_electrons=None,
            n_active_orbitals=None)

        gelist_temp = []
        iterlist_temp = []
        for i in range(10):
            ge, cnt = get_ground_energy(h2_interaction_hamil, layer)
            print('bond length: ', round(bond_length, 2), ' ground state energy: ', ge, ' iteration: ', cnt)
            gelist_temp.append(ge)
            iterlist_temp.append(cnt)
        gelist.append(np.mean(gelist_temp))
        gelist2.append(min(gelist_temp))
        gestdlist.append(np.std(gelist_temp))
        iterlist.append(np.mean(iterlist_temp))
        iterstdlist.append(np.std(iterlist_temp))
        
    vqe_ground_energies_mean.append(gelist)
    vqe_ground_energies_min.append(gelist2)
    vqe_ground_energies_std.append(gestdlist)
    vqe_iter_mean.append(np.mean(iterlist))
    vqe_iter_std.append(np.mean(iterstdlist))


-1.1441998409886938
-1.1306802112248104


KeyboardInterrupt: 